# Assignment: Webscrape Wiki page to prepare clean data for Toronto Neighbouring localities

#### [ Used following commands for installation of Requests and BeautifulSoup: 1. pip3 install requests 2. pip3 install beautifulsoup4 ]

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
#Use the Notebook to build the code to scrape the following Wikipedia page
wiki_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wiki_page_content = requests.get(wiki_link).text

raw_wiki_page_content_xml = BeautifulSoup(raw_wiki_page_content,'xml')

## Process the web content to prepare the dataframe

In [3]:
table = raw_wiki_page_content_xml.find('table')
#print(table.prettify())

Postcode      = []
Borough       = []
Neighbourhood = []

In [4]:
line_number = 1
for tr_cell in table.find_all('tr'):
    if line_number > 1 :
        column_number = 1
        Postcode_temp = None
        Borough_temp = None
        Neighbourhood_temp = None
        for td_cell in tr_cell.find_all('td'):
            if column_number == 1 :
                Postcode_temp = td_cell.text
            elif column_number == 2 :
                Borough_temp = td_cell.text
            else :
                Neighbourhood_temp = td_cell.text
            column_number = column_number + 1
        #Ignore cells with a borough that is Not assigned
        if Borough_temp != 'Not assigned':
            Postcode.append(Postcode_temp.strip())
            Borough.append(Borough_temp.strip())
            Neighbourhood_temp = Neighbourhood_temp.strip()
            if Neighbourhood_temp != 'Not assigned':
                Neighbourhood.append(Neighbourhood_temp)
            else :
                #If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
                Neighbourhood.append(Borough_temp)
    line_number = line_number + 1

In [5]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
toronto_df = pd.DataFrame({"PostalCode":Postcode,"Borough":Borough,"Neighbourhood":Neighbourhood})
#print(toronto_df.head())
#print(toronto_df.shape)

In [6]:
#More than one neighborhood can exist in one postal code area.These [multiple] rows will be combined into one row with the neighborhoods separated with a comma
toronto_df = toronto_df.groupby('PostalCode').agg({'Borough':'first', 
                             'Neighbourhood': ', '.join}).reset_index()
toronto_df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
#In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
print("Shape of the Dataframe:"+str(toronto_df.shape))

Shape of the Dataframe:(103, 3)
